# Dependencies

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Data: Wildfires

In [23]:
# Path to sqlite
database_path = "Resources/AZ_Wildfires.sqlite"

# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

In [24]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['ElementaryGeometries',
 'Fires',
 'Fires_2',
 'KNN',
 'NWCG_UnitIDActive_20170109',
 'SpatialIndex',
 'geometry_columns',
 'geometry_columns_auth',
 'geometry_columns_field_infos',
 'geometry_columns_statistics',
 'geometry_columns_time',
 'idx_Fires_Shape',
 'idx_Fires_Shape_node',
 'idx_Fires_Shape_parent',
 'idx_Fires_Shape_rowid',
 'spatial_ref_sys',
 'spatial_ref_sys_aux',
 'spatialite_history',
 'sql_statements_log',
 'sqlite_sequence',
 'views_geometry_columns',
 'views_geometry_columns_auth',
 'views_geometry_columns_field_infos',
 'views_geometry_columns_statistics',
 'virts_geometry_columns',
 'virts_geometry_columns_auth',
 'virts_geometry_columns_field_infos',
 'virts_geometry_columns_statistics']

In [25]:
# Use Inspector to print the column names and types
columns = inspector.get_columns('Fires_2')
for c in columns:
    print(c['name'], c["type"])

OBJECTID INTEGER
FOD_ID INTEGER
FPA_ID TEXT
SOURCE_SYSTEM_TYPE TEXT
SOURCE_SYSTEM TEXT
NWCG_REPORTING_AGENCY TEXT
NWCG_REPORTING_UNIT_ID TEXT
NWCG_REPORTING_UNIT_NAME TEXT
SOURCE_REPORTING_UNIT TEXT
SOURCE_REPORTING_UNIT_NAME TEXT
LOCAL_FIRE_REPORT_ID TEXT
LOCAL_INCIDENT_ID TEXT
FIRE_CODE TEXT
FIRE_NAME TEXT
ICS_209_INCIDENT_NUMBER TEXT
ICS_209_NAME TEXT
MTBS_ID TEXT
MTBS_FIRE_NAME TEXT
COMPLEX_NAME TEXT
FIRE_YEAR INTEGER
DISCOVERY_DATE REAL
DISCOVERY_DOY INTEGER
DISCOVERY_TIME TEXT
STAT_CAUSE_CODE REAL
STAT_CAUSE_DESCR TEXT
CONT_DATE REAL
CONT_DOY INTEGER
CONT_TIME TEXT
FIRE_SIZE REAL
FIRE_SIZE_CLASS TEXT
LATITUDE REAL
LONGITUDE REAL
OWNER_CODE REAL
OWNER_DESCR TEXT
STATE TEXT
COUNTY TEXT
FIPS_CODE TEXT
FIPS_NAME TEXT
Shape INTEGER
fire_discovery_date NUMERIC
fire_cont_date NUMERIC


In [26]:
# Query wildfires in the state of Arizona from the Database
data = engine.execute("SELECT FIRE_NAME, FIRE_YEAR, DISCOVERY_TIME, CONT_TIME, STAT_CAUSE_CODE, STAT_CAUSE_DESCR, FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE, LONGITUDE, OWNER_CODE, OWNER_DESCR, STATE, COUNTY, fire_discovery_date, fire_cont_date FROM Fires_2 WHERE STATE = 'AZ'")
# Create dataframe from the query
az_wildfires = pd.DataFrame(data, columns=['FIRE_NAME', 'FIRE_YEAR', 'DISCOVERY_TIME', 'CONT_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'fire_discovery_date', 'fire_cont_date'])
az_wildfires.head()

,FIRE_NAME,FIRE_YEAR,DISCOVERY_TIME,CONT_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,fire_discovery_date,fire_cont_date
0,BENHAM,2005,1713,1949,4.0,Campfire,0.5,B,35.236389,-112.216389,5.0,USFS,AZ,5,2005-05-16,2005-05-16
1,GILLESPIE,2005,1145,1400,2.0,Equipment Use,0.1,A,32.560556,-109.772500,5.0,USFS,AZ,9,2005-05-18,2005-05-18
2,CUNNINGHAM,2005,1230,1900,1.0,Lightning,0.5,B,32.685556,-109.869167,5.0,USFS,AZ,9,2005-06-26,2005-06-26
3,FRY MESA,2005,1810,1900,7.0,Arson,0.1,A,32.753611,-109.831389,5.0,USFS,AZ,9,2005-07-14,2005-07-14
4,HANKS DRAW,2005,1530,1200,1.0,Lightning,0.1,A,34.451667,-110.666111,5.0,USFS,AZ,17,2005-05-27,2005-05-28


In [27]:
# Count to detect null data in rows
az_wildfires.count()

FIRE_NAME              60986
FIRE_YEAR              71586
DISCOVERY_TIME         68538
CONT_TIME              65039
STAT_CAUSE_CODE        71586
STAT_CAUSE_DESCR       71586
FIRE_SIZE              71586
FIRE_SIZE_CLASS        71586
LATITUDE               71586
LONGITUDE              71586
OWNER_CODE             71586
OWNER_DESCR            71586
STATE                  71586
COUNTY                 24585
fire_discovery_date    71586
fire_cont_date         66355
dtype: int64

In [28]:
# Remove all null data
az_wildfires = az_wildfires.dropna()

In [29]:
# Count to show nulls have been removed
az_wildfires.count()

FIRE_NAME              20651
FIRE_YEAR              20651
DISCOVERY_TIME         20651
CONT_TIME              20651
STAT_CAUSE_CODE        20651
STAT_CAUSE_DESCR       20651
FIRE_SIZE              20651
FIRE_SIZE_CLASS        20651
LATITUDE               20651
LONGITUDE              20651
OWNER_CODE             20651
OWNER_DESCR            20651
STATE                  20651
COUNTY                 20651
fire_discovery_date    20651
fire_cont_date         20651
dtype: int64

In [30]:
# Identify unique county names and any outliers
az_wildfires['COUNTY'].unique()

array(['5', '9', '17', '7', '13', '1', '11', '21', '19', '3', '23', '25',
       '15', 'Mohave', 'Coconino', 'Yavapai', 'Maricopa', 'Graham',
       'Cochise', 'Pinal', 'Yuma', 'La Paz', 'Navajo', 'Pima', 'Gila',
       'Greenlee', 'Santa Cruz', '007', 'COCHISE', 'APACHE ', 'Apache',
       '013', '025', '005', '003', '023', '017', '019', '001', '021',
       '011', '009', 'COCONINO', 'NAVAJO', 'YUMA', 'PIMA', 'PINAL',
       'MOHAVE', 'MARICOPA', 'APACHE', 'YAVAPAI', 'SANTA CRUZ', 'GILA'],
      dtype=object)

In [31]:
# Rename any outlier county names
az_wildfires['COUNTY'] = az_wildfires['COUNTY'].replace({'1':'Apache', '001':'Apache', 'APACHE':'Apache', 'APACHE ':'Apache',
                                                         '3':'Cochise', '003':'Cochise', 'COCHISE':'Cochise',
                                                         '5':'Coconino', '005':'Coconino', 'COCONINO':'Coconino',
                                                         '7':'Gila', '007':'Gila', 'GILA':'Gila',
                                                         '9':'Graham', '009':'Graham', 'GRAHAM':'Graham',
                                                         '11':'Greenlee', '011':'Greenlee', 'GREENLEE':'Greenlee',
                                                         '12':'La Paz', '012':'La Paz', 'LA PAZ':'La Paz',
                                                         '13':'Maricopa', '013':'Maricopa', 'MARICOPA':'Maricopa',
                                                         '15':'Mohave', '015':'Mohave', 'MOHAVE':'Mohave',
                                                         '17':'Navajo', '017':'Navajo', 'NAVAJO':'Navajo',
                                                         '19':'Pima', '019':'Pima', 'PIMA':'Pima',
                                                         '21':'Pinal', '021':'Pinal', 'PINAL':'Pinal',
                                                         '23':'Santa Cruz', '023':'Santa Cruz', 'SANTA CRUZ':'Santa Cruz',
                                                         '25':'Yavapai', '025':'Yavapai', 'YAVAPAI':'Yavapai',
                                                         '27':'Yuma', '027':'Yuma', 'YUMA':'Yuma'
                                                        })

In [32]:
# Check to see that all outliers have been renamed
az_wildfires['COUNTY'].unique()

array(['Coconino', 'Graham', 'Navajo', 'Gila', 'Maricopa', 'Apache',
       'Greenlee', 'Pinal', 'Pima', 'Cochise', 'Santa Cruz', 'Yavapai',
       'Mohave', 'Yuma', 'La Paz'], dtype=object)

In [33]:
# Create SQLite as output of cleaned data
database_path = "Resources/AZWildfires.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
az_wildfires.to_sql("az_wildfires", con=engine, index=False, if_exists='append')

# Data: Droughts

In [34]:
# Import data from csv file
data_path = "Resources/USDroughts.csv"
us_droughts = pd.read_csv(data_path)
us_droughts.head()

,releaseDate,FIPS,county,state,NONE,D0,D1,D2,D3,D4,validStart,validEnd,domStatisticFormatID
0,2000-11-07,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-11-07,2000-11-13,1
1,2000-10-31,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-10-31,2000-11-06,1
2,2000-10-24,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-10-24,2000-10-30,1
3,2000-10-17,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-10-17,2000-10-23,1
4,2000-10-10,2013,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0,2000-10-10,2000-10-16,1


In [35]:
# Delete extraneous columns
del us_droughts['FIPS']
del us_droughts['domStatisticFormatID']
del us_droughts['validStart']
del us_droughts['validEnd']
us_droughts.head()

,releaseDate,county,state,NONE,D0,D1,D2,D3,D4
0,2000-11-07,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0
1,2000-10-31,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0
2,2000-10-24,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0
3,2000-10-17,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0
4,2000-10-10,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# Rename columns
us_droughts = us_droughts.rename(columns={"releaseDate":"Release_Date", "county":"County", "state":"State", "NONE":"No_drought", "D0":"Dry_conditions", "D1":"Moderate_drought", "D2":"Severe_drought", "D3":"Extreme_drought", "D4":"Exceptional_drought"})
us_droughts.head()

,Release_Date,County,State,No_drought,Dry_conditions,Moderate_drought,Severe_drought,Extreme_drought,Exceptional_drought
0,2000-11-07,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0
1,2000-10-31,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0
2,2000-10-24,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0
3,2000-10-17,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0
4,2000-10-10,Aleutians East Borough,AK,100.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Select data only for the state of Arizona
az_droughts = us_droughts.loc[(us_droughts["State"] == "AZ")]
az_droughts.head()

,Release_Date,County,State,No_drought,Dry_conditions,Moderate_drought,Severe_drought,Extreme_drought,Exceptional_drought
7605,2000-11-07,Apache County,AZ,100.00,0.00,0.00,0.00,0.0,0.0
7606,2000-10-31,Apache County,AZ,60.25,39.75,0.00,0.00,0.0,0.0
7607,2000-10-24,Apache County,AZ,49.87,50.13,38.79,0.00,0.0,0.0
7608,2000-10-17,Apache County,AZ,36.19,63.81,36.44,0.00,0.0,0.0
7609,2000-10-10,Apache County,AZ,0.00,100.00,97.95,4.49,0.0,0.0


In [38]:
# Count to detect null data in rows
az_droughts.count()

Release_Date           13185
County                 13185
State                  13185
No_drought             13185
Dry_conditions         13185
Moderate_drought       13185
Severe_drought         13185
Extreme_drought        13185
Exceptional_drought    13185
dtype: int64

In [39]:
# Remove all null data
az_droughts = az_droughts.dropna()

In [40]:
# Count to show nulls have been removed
az_droughts.count()

Release_Date           13185
County                 13185
State                  13185
No_drought             13185
Dry_conditions         13185
Moderate_drought       13185
Severe_drought         13185
Extreme_drought        13185
Exceptional_drought    13185
dtype: int64

In [41]:
# Check for unique county names and any outliers
az_droughts['County'].unique()

array(['Apache County', 'Cochise County', 'Coconino County',
       'Gila County', 'Graham County', 'Greenlee County', 'La Paz County',
       'Maricopa County', 'Mohave County', 'Navajo County', 'Pima County',
       'Pinal County', 'Santa Cruz County', 'Yavapai County',
       'Yuma County'], dtype=object)

In [42]:
# Rename county names
az_droughts['County'] = az_droughts['County'].replace({'Apache County':'Apache',
                                                         'Cochise County':'Cochise',
                                                         'Coconino County':'Coconino',
                                                         'Gila County':'Gila',
                                                         'Graham County':'Graham',
                                                         'Greenlee County':'Greenlee',
                                                         'La Paz County':'La Paz',
                                                         'Maricopa County':'Maricopa',
                                                         'Mohave County':'Mohave',
                                                         'Navajo County':'Navajo',
                                                         'Pima County':'Pima',
                                                         'Pinal County':'Pinal',
                                                         'Santa Cruz County':'Santa Cruz',
                                                         'Yavapai County':'Yavapai',
                                                         'Yuma County':'Yuma'
                                                        })

In [43]:
# Check that county names have changed
az_droughts['County'].unique()

array(['Apache', 'Cochise', 'Coconino', 'Gila', 'Graham', 'Greenlee',
       'La Paz', 'Maricopa', 'Mohave', 'Navajo', 'Pima', 'Pinal',
       'Santa Cruz', 'Yavapai', 'Yuma'], dtype=object)

In [44]:
az_droughts['Year'] = pd.DatetimeIndex(az_droughts['Release_Date']).year
az_droughts.head()

,Release_Date,County,State,No_drought,Dry_conditions,Moderate_drought,Severe_drought,Extreme_drought,Exceptional_drought,Year
7605,2000-11-07,Apache,AZ,100.00,0.00,0.00,0.00,0.0,0.0,2000
7606,2000-10-31,Apache,AZ,60.25,39.75,0.00,0.00,0.0,0.0,2000
7607,2000-10-24,Apache,AZ,49.87,50.13,38.79,0.00,0.0,0.0,2000
7608,2000-10-17,Apache,AZ,36.19,63.81,36.44,0.00,0.0,0.0,2000
7609,2000-10-10,Apache,AZ,0.00,100.00,97.95,4.49,0.0,0.0,2000


In [45]:
az_droughts['Total Percentage'] = az_droughts['Exceptional_drought'] + az_droughts['Extreme_drought'] + az_droughts['Severe_drought'] + az_droughts['Moderate_drought'] + az_droughts['Dry_conditions'] + az_droughts['No_drought']
az_droughts.head()

,Release_Date,County,State,No_drought,Dry_conditions,Moderate_drought,Severe_drought,Extreme_drought,Exceptional_drought,Year,Total Percentage
7605,2000-11-07,Apache,AZ,100.00,0.00,0.00,0.00,0.0,0.0,2000,100.00
7606,2000-10-31,Apache,AZ,60.25,39.75,0.00,0.00,0.0,0.0,2000,100.00
7607,2000-10-24,Apache,AZ,49.87,50.13,38.79,0.00,0.0,0.0,2000,138.79
7608,2000-10-17,Apache,AZ,36.19,63.81,36.44,0.00,0.0,0.0,2000,136.44
7609,2000-10-10,Apache,AZ,0.00,100.00,97.95,4.49,0.0,0.0,2000,202.44


In [46]:
# All data defaults to No Drought
az_droughts['No_drought'] = 1

# Select data only for Dry Conditions and update
dry = az_droughts.loc[(az_droughts["Dry_conditions"] <= 50)]
dry['Dry_conditions'] = 0
az_droughts['Dry_conditions'].update(dry['Dry_conditions'])

# Select data only for Dry Conditions and update
dry = az_droughts.loc[(az_droughts["Dry_conditions"] > 50)]
dry['Dry_conditions'] = 1
dry['No_drought'] = 0
az_droughts['Dry_conditions'].update(dry['Dry_conditions'])
az_droughts['No_drought'].update(dry['No_drought'])

# Select data only for Moderate Drought and update
mod = az_droughts.loc[(az_droughts["Moderate_drought"] <= 50)]
mod['Moderate_drought'] = 0
az_droughts['Moderate_drought'].update(mod['Moderate_drought'])

# Select data only for Moderate Drought and update   
mod = az_droughts.loc[(az_droughts["Moderate_drought"] > 50)]
mod['Moderate_drought'] = 1
mod['Dry_conditions'] = 0
mod['No_drought'] = 0
az_droughts['Moderate_drought'].update(mod['Moderate_drought'])
az_droughts['Dry_conditions'].update(mod['Dry_conditions'])
az_droughts['No_drought'].update(mod['No_drought'])

# Select data only for Severe Drought and update
severe = az_droughts.loc[(az_droughts["Severe_drought"] <= 50)]
severe['Severe_drought'] = 0
az_droughts['Severe_drought'].update(severe['Severe_drought'])

# Select data only for Severe Drought and update
severe = az_droughts.loc[(az_droughts["Severe_drought"] > 50)]
severe['Severe_drought'] = 1
severe['Moderate_drought'] = 0
severe['Dry_conditions'] = 0
severe['No_drought'] = 0
az_droughts['Severe_drought'].update(severe['Severe_drought'])
az_droughts['Moderate_drought'].update(severe['Moderate_drought'])
az_droughts['Dry_conditions'].update(severe['Dry_conditions'])
az_droughts['No_drought'].update(severe['No_drought'])

# Select data only for Extreme Drought and update  
extreme = az_droughts.loc[(az_droughts["Extreme_drought"] <= 50)]
extreme['Extreme_drought'] = 0
az_droughts['Extreme_drought'].update(extreme['Extreme_drought'])

# Select data only for Extreme Drought and update  
extreme = az_droughts.loc[(az_droughts["Extreme_drought"] > 50)]
extreme['Extreme_drought'] = 1
extreme['Severe_drought'] = 0
extreme['Moderate_drought'] = 0
extreme['Dry_conditions'] = 0
extreme['No_drought'] = 0
az_droughts['Extreme_drought'].update(extreme['Extreme_drought'])
az_droughts['Severe_drought'].update(severe['Severe_drought'])
az_droughts['Moderate_drought'].update(extreme['Moderate_drought'])
az_droughts['Dry_conditions'].update(extreme['Dry_conditions'])
az_droughts['No_drought'].update(extreme['No_drought'])

# Select data only for Exceptional Drought and update
exceptional = az_droughts.loc[(az_droughts["Exceptional_drought"] <= 50)]
exceptional['Exceptional_drought'] = 0
az_droughts['Exceptional_drought'].update(exceptional['Exceptional_drought'])

# Select data only for Exceptional Drought and update
exceptional = az_droughts.loc[(az_droughts["Exceptional_drought"] > 50)]
exceptional['Exceptional_drought'] = 1
exceptional['Extreme_drought'] = 0
exceptional['Severe_drought'] = 0
exceptional['Moderate_drought'] = 0
exceptional['Dry_conditions'] = 0
exceptional['No_drought'] = 0
az_droughts['Exceptional_drought'].update(exceptional['Exceptional_drought'])
az_droughts['Extreme_drought'].update(exceptional['Extreme_drought'])
az_droughts['Severe_drought'].update(exceptional['Severe_drought'])
az_droughts['Moderate_drought'].update(exceptional['Moderate_drought'])
az_droughts['Dry_conditions'].update(exceptional['Dry_conditions'])
az_droughts['No_drought'].update(exceptional['No_drought'])

# Display data
az_droughts.head()

<ipython-input-46-30ca51f54bea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dry['Dry_conditions'] = 0
<ipython-input-46-30ca51f54bea>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dry['Dry_conditions'] = 1
<ipython-input-46-30ca51f54bea>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

,Release_Date,County,State,No_drought,Dry_conditions,Moderate_drought,Severe_drought,Extreme_drought,Exceptional_drought,Year,Total Percentage
7605,2000-11-07,Apache,AZ,1,0.0,0.0,0.0,0.0,0.0,2000,100.00
7606,2000-10-31,Apache,AZ,1,0.0,0.0,0.0,0.0,0.0,2000,100.00
7607,2000-10-24,Apache,AZ,0,1.0,0.0,0.0,0.0,0.0,2000,138.79
7608,2000-10-17,Apache,AZ,0,1.0,0.0,0.0,0.0,0.0,2000,136.44
7609,2000-10-10,Apache,AZ,0,0.0,1.0,0.0,0.0,0.0,2000,202.44


In [47]:
# Delete extraneous columns
del az_droughts['Release_Date']
del az_droughts['Total Percentage']
az_droughts.head()

,County,State,No_drought,Dry_conditions,Moderate_drought,Severe_drought,Extreme_drought,Exceptional_drought,Year
7605,Apache,AZ,1,0.0,0.0,0.0,0.0,0.0,2000
7606,Apache,AZ,1,0.0,0.0,0.0,0.0,0.0,2000
7607,Apache,AZ,0,1.0,0.0,0.0,0.0,0.0,2000
7608,Apache,AZ,0,1.0,0.0,0.0,0.0,0.0,2000
7609,Apache,AZ,0,0.0,1.0,0.0,0.0,0.0,2000


In [48]:
#final = az_droughts.groupby(['County']).count()

final = pd.DataFrame(az_droughts.value_counts().reset_index().values, columns=["County", "State", "No_drought", "Dry_conditions", "Moderate_drought", "Severe_drought", "Extreme_drought", "Exceptional_drought", "Year", "Count"])
final

,County,State,No_drought,Dry_conditions,Moderate_drought,Severe_drought,Extreme_drought,Exceptional_drought,Year,Count
0,Graham,AZ,0,0.0,0.0,1.0,0.0,0.0,2004,52
1,La Paz,AZ,0,1.0,0.0,0.0,0.0,0.0,2015,52
2,Maricopa,AZ,0,0.0,1.0,0.0,0.0,0.0,2015,52
3,Maricopa,AZ,1,0.0,0.0,0.0,0.0,0.0,2001,52
4,Navajo,AZ,1,0.0,0.0,0.0,0.0,0.0,2001,52
...,...,...,...,...,...,...,...,...,...,...
743,Yuma,AZ,0,0.0,1.0,0.0,0.0,0.0,2007,1
744,Pinal,AZ,0,0.0,1.0,0.0,0.0,0.0,2008,1
745,Yavapai,AZ,0,0.0,1.0,0.0,0.0,0.0,2006,1
746,Greenlee,AZ,0,0.0,0.0,1.0,1.0,0.0,2003,1


In [49]:
# All data defaults to No Drought
final['Condition'] = 'No_drought'

# Select data only for Dry Conditions and update
dry = final.loc[(final["Dry_conditions"] == 1)]
dry['Condition'] = 'Dry_conditions'
final['Condition'].update(dry['Condition'])

# Select data only for Moderate Drought and update
mod = final.loc[(final["Moderate_drought"] == 1)]
mod['Condition'] = 'Moderate_drought'
final['Condition'].update(mod['Condition'])

# Select data only for Severe Drought and update
severe = final.loc[(final["Severe_drought"] == 1)]
severe['Condition'] = 'Severe_drought'
final['Condition'].update(severe['Condition'])

# Select data only for Extreme Drought and update  
extreme = final.loc[(final["Extreme_drought"] == 1)]
extreme['Condition'] = 'Extreme_drought'
final['Condition'].update(extreme['Condition'])

# Select data only for Exceptional Drought and update
exceptional = final.loc[(final["Exceptional_drought"] == 1)]
exceptional['Condition'] = 'Exceptional Drought'
final['Condition'].update(exceptional['Condition'])

# Display data
final.head()

<ipython-input-49-d0fbbb2ce243>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dry['Condition'] = 'Dry_conditions'
<ipython-input-49-d0fbbb2ce243>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod['Condition'] = 'Moderate_drought'
<ipython-input-49-d0fbbb2ce243>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,County,State,No_drought,Dry_conditions,Moderate_drought,Severe_drought,Extreme_drought,Exceptional_drought,Year,Count,Condition
0,Graham,AZ,0,0.0,0.0,1.0,0.0,0.0,2004,52,Severe_drought
1,La Paz,AZ,0,1.0,0.0,0.0,0.0,0.0,2015,52,Dry_conditions
2,Maricopa,AZ,0,0.0,1.0,0.0,0.0,0.0,2015,52,Moderate_drought
3,Maricopa,AZ,1,0.0,0.0,0.0,0.0,0.0,2001,52,No_drought
4,Navajo,AZ,1,0.0,0.0,0.0,0.0,0.0,2001,52,No_drought


In [50]:
# Delete extraneous columns
del final['No_drought']
del final['Dry_conditions']
del final['Moderate_drought']
del final['Severe_drought']
del final['Extreme_drought']
del final['Exceptional_drought']
final.head()

,County,State,Year,Count,Condition
0,Graham,AZ,2004,52,Severe_drought
1,La Paz,AZ,2015,52,Dry_conditions
2,Maricopa,AZ,2015,52,Moderate_drought
3,Maricopa,AZ,2001,52,No_drought
4,Navajo,AZ,2001,52,No_drought


In [51]:
# Create SQLite as output of cleaned data
database_path = "Resources/AZWildfires.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
final.to_sql("az_droughts", con=engine, index=False, if_exists='append')

In [52]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['az_droughts', 'az_wildfires', 'sqlite_sequence']

In [53]:
database_path = "Resources/AZWildfires.sqlite"
engine = create_engine(f"sqlite:///{database_path}")


# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['az_droughts', 'az_wildfires']